In [1]:
#**********Realsense bag file -> depth, grayscale, and edge*********
# This notebook assumes several things, firstly that the bag files are recorded ~1-1.5m from the cow's back.
# This is so that the thresholds used to find the cow in the frame can be hardcoded, if you find that there is too much/too little of the cow in frame,
# they will have to be adjusted.

# Next, the code assumes the following file architecture: working directory (that this notebook is in)\dataset folder\BCS class each bagfile belongs to\bagfiles themselves
# The output file architecture: dataset_folder_processed\BCS_class\frame_number_DGE.tif
# The dataloader for the transformer assumes that each image is independent of each other image, so what cow each frame came from is irrelevent.
# This leaves images belonging to the same class being mixed together.

# Parameters for each operation are changed in the last cell of the notebook, with the defaults listed beside them

# ****The class folders must be deleted between runs, as this code does not delete previous rundata, it just assigns images names based on the total number of
# images in the class folder, so deleting images and then running the code again will not generate a fresh dataset, and will instead overwrite some images that have 
# conflicting names

#Lastly, this expects all bag files to be 640x480 resolution, and filmed with NO postprocessing. FPS doesn't matter so long as the cow is actually cpatured at
# some point, although 30 fps seems to be too much for the USB 2.0 cable to use, and it corrupts the resulting bagfile, 10-15 fps is recommended.

import pyrealsense2.pyrealsense2 as rs
import numpy as np
import cv2
import time
import os
import datetime
import skimage.filters
import pandas as pd
import glob
import imageio

# Change this to the directory containing the class folders and bag files
bag_files_dir = 'BCS_classes_with_bag_files'

In [2]:
def image_preprocessing(depth_img, background):
    # [:, int((848-640)/2):int(-(848-640)/2)]
    depth_img = background - depth_img

    otsu_thresh = skimage.filters.threshold_otsu(depth_img)
    img_post = np.where(depth_img < otsu_thresh, depth_img, 0)

    img_post = np.where(img_post > 600, depth_img, 0)

    img_mask = np.where(img_post != 0, 1, 0)
    img_seg = skimage.measure.label(img_mask, background=0, connectivity=2)
    #assert( img_seg.max() != 0 )
    if img_seg.max() != 0:
        mask = img_seg == np.argmax(np.bincount(img_seg.flat)[1:])+1
    else:
        mask = np.zeros(np.shape(depth_img))

    depth_img = np.where(mask != 0, depth_img, 0)
    try:
        depth_img = (depth_img/np.max(depth_img)*255).astype('uint8')
    except:
        pass

    return depth_img, mask


In [3]:
# Modifications to image preprocessing should be done here.

def extract_tiff_from_bag(filename, thresh_lower, thresh_higher, pixel_thresh_top, pixel_thresh_lower, 
                        pixel_thresh_total_non_zero, spatial_mag, spatial_smooth_alpha, spatial_smooth_delta, 
                        spatial_hole_fill, temporal_smooth_alpha, temporal_smooth_delta, temporal_hole_fill):
    BCS_class = os.path.dirname(filename)[-1]
    parent_dir = os.path.basename(os.path.dirname(os.path.dirname(filename)))
    num_pics_in_class = len(glob.glob(parent_dir+'_processed'+'\\'+BCS_class+'\\*'))
    length = 480


    try:
        pipeline = rs.pipeline()
        config = rs.config()
        config.enable_device_from_file(filename, False)
        pipeline.start(config)
    except RuntimeError:
        return


    spatial = rs.spatial_filter()
    spatial.set_option(rs.option.filter_magnitude, spatial_mag)
    spatial.set_option(rs.option.filter_smooth_alpha, spatial_smooth_alpha)
    spatial.set_option(rs.option.filter_smooth_delta, spatial_smooth_delta)
    spatial.set_option(rs.option.holes_fill, spatial_hole_fill)

    #this temporal filtering is added 1/25/22, so it might be the source of things breaking
    temporal = rs.temporal_filter()
    temporal.set_option(rs.option.filter_smooth_alpha, temporal_smooth_alpha)
    temporal.set_option(rs.option.filter_smooth_delta, temporal_smooth_delta)
    temporal.set_option(rs.option.holes_fill, temporal_hole_fill)

    thresh = rs.threshold_filter(min_dist = thresh_lower, max_dist = thresh_higher)
    
    align_to = rs.stream.color
    align = rs.align(align_to)


    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    color_image=np.asanyarray(color_frame.get_data())
    thresh_frame = thresh.process(depth_frame)
    spatial_depth = spatial.process(thresh_frame)
    filtered_depth = temporal.process(spatial_depth)
    background_image = np.asanyarray(filtered_depth.get_data())
    if np.shape(background_image)[1] != np.shape(color_image)[1]:
                    #print(np.shape(background_image)[1])
                    #print(np.shape(color_image)[1])
                    new_depth = np.zeros(np.shape(background_image[:, 104:-104]))
                    #print(np.shape(new_depth)[1])
                    new_depth=background_image[:, 104:-104]
                    background_image=new_depth
                    #print(np.shape(background_image)[1])


    stop = True
    while True:
        try:
            #getting frames from the camera
            frames = pipeline.wait_for_frames(100)
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            thresh_frame = thresh.process(depth_frame)
            spatial_depth = spatial.process(thresh_frame)
            filtered_depth = temporal.process(spatial_depth)
            depth_image = np.asanyarray(filtered_depth.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            if np.shape(depth_image)[1] != np.shape(color_image)[1]:
                    #print(np.shape(depth_image)[1])
                    #print(np.shape(color_image)[1])
                    new_depth = np.zeros(np.shape(depth_image[:, 104:-104]))
                    #print(np.shape(new_depth)[1])
                    new_depth=depth_image[:, 104:-104]
                    depth_image=new_depth
                    #print(np.shape(depth_image)[1])



            cropped_depth, mask = image_preprocessing(depth_image, background_image)
            upper_pixels = np.sum(np.where(cropped_depth[:100, :] != 0, 1, 0))
            lower_pixels = np.sum(np.where(cropped_depth[length-20:length, :] != 0, 1, 0))
            total_nonzero_pixels = np.sum(np.where(cropped_depth != 0, 1, 0))
            if (lower_pixels < pixel_thresh_lower) and (upper_pixels > pixel_thresh_top) and (total_nonzero_pixels > pixel_thresh_total_non_zero):#was 15k and 14k
                
                cropped_depth = (cropped_depth/np.max(cropped_depth)*255).astype('uint8')

                # Processing not done by pyrealsense functions can be done on the variables below.
                
                cropped_color = color_image
                cropped_color[:, :, 0] = np.where(mask==True, color_image[:, :, 0], 0)
                cropped_color[:, :, 1] = np.where(mask==True, color_image[:, :, 1], 0)
                cropped_color[:, :, 2] = np.where(mask==True, color_image[:, :, 2], 0)

                img_DGE = np.zeros(np.shape(cropped_color))
                img_DGE[:, :, 0] = cropped_depth
                img_DGE[:, :, 1] = skimage.color.rgb2gray(cropped_color)
                mask_dilation = skimage.morphology.binary_dilation(mask, skimage.morphology.ball(4, dtype=bool)[::, ::, 0], out=None)
                img_DGE[:, :, 2] = skimage.feature.canny(cropped_depth, sigma=1.0).astype('uint8')*255*mask_dilation

                imageio.imwrite(parent_dir+'_processed\\'+str(BCS_class)+'\\'+f"{num_pics_in_class}_DGE.tif", img_DGE.astype('uint8'), 'TIFF')
                num_pics_in_class+=1

            time.sleep(0.01)
            
        except RuntimeError:
            pipeline.stop()
            stop = False
            print(filename)
            break
    
    if stop == True:
        pipeline.stop()
        print(filename)
    return

In [4]:
filenames = glob.glob(bag_files_dir+'\\*\\*.bag')

dest_dir = bag_files_dir+'_processed'
try:
    os.mkdir(dest_dir)
except FileExistsError:
    pass

for i in range(1, 10):
    try:
        os.mkdir(dest_dir+'\\'+str(i))
    except FileExistsError:
        pass

for bag_file in filenames:
    try:
        extract_tiff_from_bag(bag_file, 
                            thresh_lower = 0.1, #0.1
                            thresh_higher = 3.0, #3.0
                            pixel_thresh_top = 10000, #10000
                            pixel_thresh_lower = 400, #400
                            pixel_thresh_total_non_zero = 25000, #25000
                            spatial_mag = 5, #5
                            spatial_smooth_alpha = 0.5, #0.5
                            spatial_smooth_delta = 20, #20
                            spatial_hole_fill = 2, #2
                            temporal_smooth_alpha = 0.15, #0.15
                            temporal_smooth_delta = 50, #50
                            temporal_hole_fill = 3, #3
                            )
        #print(filenames[i], 'done')
    except Exception as e:
        print(bag_file, 'failed')
        print(e)

February_Data\3\Cow_8_2023.bag
February_Data\4\Cow_10_0014.bag
February_Data\4\Cow_11_0075.bag
February_Data\4\Cow_2_6016.bag
February_Data\4\Cow_3_6018.bag
February_Data\4\Cow_5_8041.bag
February_Data\5\Cow_12_6006.bag
February_Data\5\Cow_13_8001.bag
February_Data\5\Cow_14_1044.bag
February_Data\5\Cow_4_8036.bag
February_Data\5\Cow_6_5029.bag
February_Data\5\Cow_7_6022.bag
February_Data\5\Cow_9_NOTG.bag
February_Data\misc\Background.bag


C:\Users\zacha\AppData\Local\Temp/ipykernel_12568/2815287608.py:20: RuntimeWarning: invalid value encountered in true_divide
  depth_img = (depth_img/np.max(depth_img)*255).astype('uint8')


February_Data\misc\Background2.bag
February_Data\misc\shwelemia.bag failed
The directory 'c:\\Users\\zacha\\Documents\\Precision Ranching Code\\February_Data_processed\\c' does not exist
February_Data\misc\test.bag
